In [1]:
from XQuant import BackTestRunner, BackTestOptions, BARRA, Basic, BARRA
from XQuant import Strategy, DataAPI, Config, Tools, DataReady, EnhancingDividend, Formatter, TradeDate
import pandas as pd
import numpy as np
import statsmodels.api as sm

ed = Basic(begin='20190101')
df = ed.EBITDA
df

ticker,000001.SZ,000002.SZ,000003.SZ,000004.SZ,000005.SZ,000006.SZ,000007.SZ,000008.SZ,000009.SZ,000010.SZ,...,900947.SH,900948.SH,900949.SH,900950.SH,900951.SH,900952.SH,900953.SH,900955.SH,900956.SH,900957.SH
2019-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-09-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-09-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-09-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
Tools.search_keyword('depr_oga_cba', limit=100, fuzzy=False)

2023-09-17 11:43:03.185 | INFO     | XQuant.Utils:search_keyword:644 - 缓存文件被保存至E:\yuankangrui\XQuant\XQuant\Temp\attrs.json


{'depr_oga_cba': ['fundamentals_cashflow',
  'financial_sheet',
  'market_financial_sheet']}

In [2]:
Config.datatables['deriv_finance_indicator']

{'assets': 'gm_stock',
 'description': '',
 'date_column': 'pub_date',
 'ticker_column': 'symbol'}

In [6]:
df = DataAPI.get_data('deriv_finance_indicator', begin='20220101')
df

,symbol,pub_date,end_date,CASHCONVCYCLE,NPTONOCONMS,REPS,COMPDEPRT,CAPEXTODEPANDAMOR,CAPIMORTCONMS,ASSLIABRT,...,FINLEXPRT,TRIEXPRT,EQUTURNRT,TDEBT,CURRENTRT,TCEXPRT,SGPMARGIN,FAPROPORTION,NDEBTTOEQU,EBITDAMARGIN
0,SHSE.600027,2022-03-26 00:00:00+08:00,2021-12-31 00:00:00+08:00,1.023362,73.495872,1.626116,4.526747,1.204799,257.127716,66.363045,...,4.090321,5.817615,2.347045,1.187862e+11,0.450464,112.008926,-6.191312,62.783783,2.860219,6.497436
1,SHSE.600020,2022-04-16 00:00:00+08:00,2021-12-31 00:00:00+08:00,356.699707,100.019814,3.144979,4.833482,0.225396,328.032379,73.510208,...,23.955301,27.929461,0.540942,3.080167e+10,0.884139,86.723579,41.205879,26.348600,2.792991,68.808998
2,SHSE.600025,2022-04-23 00:00:00+08:00,2021-12-31 00:00:00+08:00,26.269999,92.944397,0.784875,3.081494,1.049978,237.045258,58.776878,...,16.706619,18.774565,0.403878,8.135342e+10,0.162074,64.145988,54.628574,90.376961,1.553422,80.376152
3,SHSE.600016,2022-03-30 00:00:00+08:00,2021-12-31 00:00:00+08:00,0.000000,98.645737,6.737631,10.621029,1.277845,0.000000,91.563972,...,0.000000,0.000000,0.357638,7.138800e+11,0.000000,0.000000,21.410629,0.798327,0.000000,0.000000
4,SHSE.600029,2022-03-31 00:00:00+08:00,2021-12-31 00:00:00+08:00,-26.239115,109.917358,0.154708,5.023830,1.809609,338.396332,73.913757,...,3.959899,12.490653,1.484265,9.246000e+10,0.341165,115.033844,-2.543189,28.121244,1.044412,2.430049
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25597,SZSE.301389,2023-04-27 00:00:00+08:00,2023-03-31 00:00:00+08:00,208.308182,100.000000,1.822629,0.000000,0.000000,4.929801,2.254595,...,-27.707092,-15.660125,0.020378,5.342092e+06,46.641079,37.750450,46.589424,0.000000,-0.870085,0.000000
25598,SZSE.301408,2023-04-24 00:00:00+08:00,2023-03-31 00:00:00+08:00,97.920464,101.115501,1.110476,0.000000,0.000000,54.015827,48.580486,...,0.559261,27.261366,0.602799,9.121777e+08,1.783679,94.747513,32.513851,0.000000,-0.214480,0.000000
25599,SZSE.301387,2023-04-28 00:00:00+08:00,2023-03-31 00:00:00+08:00,102.311928,98.784042,5.746607,0.000000,0.000000,104.909904,50.836819,...,4.182614,14.804293,0.349223,3.751635e+08,1.059171,82.311615,32.492676,0.000000,0.455407,0.000000
25600,SZSE.301386,2023-04-20 00:00:00+08:00,2023-03-31 00:00:00+08:00,139.507782,100.000000,2.586305,0.000000,0.000000,10.945191,10.185361,...,-0.872476,7.549861,0.108673,5.292088e+07,8.936991,79.420853,28.129011,0.000000,-0.723563,0.000000


In [4]:
df = df.pivot(index='pub_date', values='NVALCHGIT', columns='symbol')
df

symbol,SHSE.600000,SHSE.600004,SHSE.600006,SHSE.600007,SHSE.600008,SHSE.600009,SHSE.600010,SHSE.600011,SHSE.600012,SHSE.600015,...,SZSE.301387,SZSE.301388,SZSE.301389,SZSE.301391,SZSE.301396,SZSE.301398,SZSE.301408,SZSE.301419,SZSE.301429,SZSE.301439
pub_date,,,,,,,,,,,,,,,,,,,,,
2022-01-05 00:00:00+08:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-06 00:00:00+08:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-07 00:00:00+08:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-10 00:00:00+08:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-12 00:00:00+08:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-27 00:00:00+08:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1282670.125,NaN,NaN,13016.69043,NaN,NaN,-1107075.125,NaN
2023-04-28 00:00:00+08:00,NaN,NaN,NaN,NaN,-9579364.0,NaN,-68751776.0,NaN,-12178203.0,8.552000e+09,...,-668498.875,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
2023-04-29 00:00:00+08:00,9.214000e+09,NaN,126095072.0,768509.0,NaN,176486512.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.groupby(pd.Grouper(freq="Q")).mean().rolling(window=4).sum()

In [ ]:
from typing import Sequence
import statsmodels.api as sm
def ACF(series:Sequence,  nlags: int = 10, **kwargs):
    """
    根据弱平稳性质$σ_s=σ_t$，有$ρ_k=(∑_t^T((X_t-\overline{X_t}))(X_s-\overline{X_s }))/(∑_t^Tσ_t^2 )$，则$ρ_1,ρ_2,…$称为时间序列$X_t$的样本自相关系数ACF。
    自相关系数呈指数的速度锐减，始终有非零取值，不会在k大于某个常数之后就恒等于零，这个性质就是平稳$AR(p) $模型的自相关系数$ρ_k$具有拖尾性。
    当自相关函数中所有的值都为0时，我们认为该序列是完全不相关的；因此，我们经常需要检验多个自相关系数是否为0。<br>
    混成检验：<br>
    原假设$H_0：ρ_1=⋯=ρ_m=0$<br>
    统计量：$Q(m)=T(T+2)∑_k^m{\frac{(ρ_k)^2}{(T-k)}}$<br>
    $Q(m)$渐进服从自由度为m的$χ^2$分布，如果$Q(m)>χ_α^2$，即Q(m)统计量的$pvalue≤显著性水平α，则拒绝H_0。$<br>
    :param series:
    :param nlags:
    :param kwargs:
    :return:
    """
    if not isinstance(series, np.ndarray):
        series = np.array(series)
    acf,q,p= sm.tsa.acf(x=series, nlags=nlags, qstat=True)
    out = np.c_[range(1,nlags+1), acf[1:], q, p]
    output=pd.DataFrame(out, columns=['lag', "AC", "Q", "P-value"])
    output = output.set_index('lag')
    if all(p < 0.05):
        print("所有的p-value都小于0.05，拒绝原假设H_0，认为该序列是序列相关的")
    return output

ACF(series=ed.close.iloc[:,0])

In [ ]:
import statsmodels.tsa.api as smt
import statsmodels.formula.api as smf
from statsmodels.tsa.ar_model import AutoReg as AR
import scipy.stats as scs
from arch import arch_model
import matplotlib.pyplot as plt

In [ ]:
def tsplot(
        y:Sequence,
        diff: bool = False,
        nlags:int=30, figsize:tuple[int, int] =(8, 8), style='bmh', **kwargs):
    if not isinstance(y, pd.Series):
        y = pd.Series(y)
    if diff:
        y = y.diff(1).dropna()

    with plt.style.context(style):
        fig = plt.figure(figsize=figsize)
        layout = (3, 2)
        ts_ax = plt.subplot2grid(layout, (0, 0), colspan=2)
        acf_ax = plt.subplot2grid(layout, (1, 0))
        pacf_ax = plt.subplot2grid(layout, (1, 1))
        qq_ax = plt.subplot2grid(layout, (2, 0))
        pp_ax = plt.subplot2grid(layout, (2, 1))

        y.plot(ax=ts_ax)
        ts_ax.set_title('Time Series Analysis')
        smt.graphics.plot_acf(y, lags=nlags, ax=acf_ax, alpha=0.05,title='ACF')
        smt.graphics.plot_pacf(y, lags=nlags, ax=pacf_ax, alpha=0.05,title='PACF')
        sm.qqplot(y, line='s', ax=qq_ax)
        qq_ax.set_title('QQ Plot')
        scs.probplot(y, sparams=(y.mean(), y.std()), plot=pp_ax)
        plt.tight_layout()
        plt.show()

tsplot(ed.close.iloc[:,0], lags=30, diff=True)

In [ ]:
est_order = statsmodels.tsa.ar_model.ar_select_order(ed.close.iloc[:,0].values, maxlag=30, ic='aic')
est_order.ar_lags

In [ ]:
import statsmodels
max_lag = 30
mdl = sm.tsa.ARIMA(ed.close.iloc[:,0].values, order=(16, 1, 1)).fit()
print(mdl.summary())